In [1]:
import os
import h5py
import numpy as np
from matplotlib import image
import matplotlib.pyplot as plot
import numpy as np
import json
import io
import unicodecsv as csv
import tensorflow as tf
import gc
#import PIL
#from PIL import Image
#print('Pillow Version:', PIL.__version__)

import numpy as np

from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Dropout
from tensorflow.keras.layers import Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, LSTM
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
#from keras.applications.imagenet_utils import preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras import optimizers



import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import plot_model
#from resnets_utils import *
from tensorflow.keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import tensorflow.keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)


import numpy as np
import os
import sys
import time
import pandas as pd 
#from tqdm._tqdm_notebook import tqdm_notebook
import pickle

cutoff = 10.0

# from keras.wrappers.scikit_learn import KerasClassifier
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#import logging


/home/aj/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aj/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aj/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aj/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
INPUT_PATH = "/home/aj/eurusd"
TIMESTEPS = 100
BATCHSIZE = 100
NUMRECORDS = 10398
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [3]:
eurusddata = pd.read_csv(os.path.join(INPUT_PATH, "EURUSDDAILY.csv"), engine='python')
print(eurusddata.shape)
print(eurusddata.columns)

#display(eurusddata.head(5))
#display(eurusddata.tail(1451))



dim_0 = eurusddata.shape[0] - TIMESTEPS 
dim_1 = eurusddata.shape[1] -2
print(dim_0)
print(dim_1)
dim_1 = 1

(10398, 6)
Index(['Date', 'Price', 'Open', 'High', 'Low', 'Change %'], dtype='object')
10298
4


In [4]:
#ffg
pricearray = np.zeros((10398,1))

#gg
eurusddata1 = eurusddata.iloc[::-1]
display(eurusddata1.head(1451))
#display(eurusddata1.tail(5))


# IMPORTANT: Data flat from record# 0 till record# 1450



,Date,Price,Open,High,Low,Change %
10397,8-Aug-80,1.4599,1.4599,1.4599,1.4599,-0.08%
10396,11-Aug-80,1.4592,1.4592,1.4592,1.4592,-0.05%
10395,12-Aug-80,1.4607,1.4607,1.4607,1.4607,0.10%
10394,13-Aug-80,1.4620,1.4620,1.4620,1.4620,0.09%
10393,14-Aug-80,1.4571,1.4571,1.4571,1.4571,-0.34%
...,...,...,...,...,...,...
8951,30-Apr-86,0.9938,0.9938,0.9938,0.9938,0.09%
8950,1-May-86,0.9793,0.9793,0.9793,0.9793,-1.46%
8949,2-May-86,0.9790,0.9790,0.9790,0.9790,-0.03%
8948,6-May-86,0.9747,0.9747,0.9747,0.9747,-0.44%


        
for i in range(dim_0+TIMESTEPS):
            
            #print(TIMESTEPS+j+1)
        pricearray[i] = eurusddata1.iloc[0+i, 3:4]

In [5]:
for i in range(dim_0+TIMESTEPS):
            
            #print(TIMESTEPS+j+1)
        pricearray[i] = eurusddata1.iloc[0+i, 3:4]

In [6]:

print(pricearray[1449])
offsetmin = np.amin(pricearray)
maxinrange = np.amax(pricearray)
print(offsetmin)
print(maxinrange)

[0.9747]
0.6444
1.6039


In [7]:
print(pricearray[5000])
print(pricearray[5100])
pricearray = pricearray/(maxinrange + 0.0001)
print(pricearray[5000])
print(pricearray[5100])

[1.01]
[0.8999]
[0.62967581]
[0.56103491]


In [8]:
offsetmin = np.amin(pricearray)
maxinrange = np.amax(pricearray)
print(offsetmin)
print(maxinrange)

0.4017456359102244
0.9999376558603491


In [9]:
#for i in range(len(pricearray)):
   # if pricearray[i] == 0.0:
     #   print(i)

In [10]:
#fsdfsa

In [11]:
x = np.zeros((dim_0-1450, 1, TIMESTEPS ))
y = np.zeros((dim_0-1450, dim_1))

In [12]:
x.shape


(8848, 1, 100)

In [13]:
y.shape

(8848, 1)

In [14]:
'''
arr = eurusddata1.to_numpy()
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [120, 80]
x = np.arange(10398)

y1 = arr[:, 1]
y2 = arr[:, 2]
y3 = arr[:, 3]
y4 = arr[:, 4]
print(y1)
print(y2)
print(y3)
print(y4)


#plt.plot(x, y1, label="John")

#plt.plot(x, y2, label="David")

#plt.plot(x, y3, label="Tom")

plt.plot(x, y3 - y4, label="Tomato")
'''
#dsada
eurusddata1 = eurusddata.iloc[::-1]
#display(eurusddata1.head(61))
#display(eurusddata1.tail(5))

for i in range(dim_0-1450):
            
            #print(TIMESTEPS+j+1)
            s = pricearray[1450+i:1450+TIMESTEPS+i, :]
            x[i] = s.reshape(1,TIMESTEPS)
            y[i] = pricearray[1450+TIMESTEPS+i]

In [15]:


print(x[0])
print(y[0])

print(x[1])
print(y[1])

print(x[10296-1450])
print(y[10296-1450])

print(x[10297-1450])
print(y[10297-1450])


[[0.61533666 0.61415212 0.61683292 0.6165212  0.61147132 0.61265586
  0.60592269 0.60118454 0.60473815 0.5973192  0.59264339 0.59052369
  0.5872818  0.58908978 0.59145885 0.58703242 0.580798   0.57842893
  0.58996259 0.59201995 0.59706983 0.60087282 0.59937656 0.60355362
  0.60592269 0.60679551 0.60473815 0.60885287 0.6026808  0.60174564
  0.6        0.59613466 0.59052369 0.60174564 0.60473815 0.60174564
  0.60561097 0.60916459 0.61359102 0.61209476 0.61564838 0.61477556
  0.61708229 0.61564838 0.61477556 0.61265586 0.61003741 0.60735661
  0.61359102 0.6165212  0.61770574 0.61502494 0.62331671 0.62244389
  0.62094763 0.62125935 0.61683292 0.61976309 0.62418953 0.62537406
  0.62861596 0.63154613 0.63185786 0.6292394  0.63216958 0.63453865
  0.63453865 0.63778055 0.63541147 0.63453865 0.63690773 0.63485037
  0.63690773 0.63865337 0.64164589 0.63983791 0.64133416 0.64014963
  0.63952618 0.63865337 0.63983791 0.64220698 0.64488778 0.64457606
  0.64513716 0.64600998 0.64488778 0.63778055 0.

In [16]:
'''
x1 = x[:,:,1]
y1 = y[:,:,1]
print(x1)
print(y1)
'''

'\nx1 = x[:,:,1]\ny1 = y[:,:,1]\nprint(x1)\nprint(y1)\n'

In [17]:
'''
x = x1 - x0
y = y1 - y0
for i in range(1):
           print(x[i])
           print(y[i])

j=1

print(x[3015])
print(y[3015])
x =  x[3015:,:]
y =  y[3015:,:]
'''


'\nx = x1 - x0\ny = y1 - y0\nfor i in range(1):\n           print(x[i])\n           print(y[i])\n\nj=1\n\nprint(x[3015])\nprint(y[3015])\nx =  x[3015:,:]\ny =  y[3015:,:]\n'

In [18]:
x.shape

(8848, 1, 100)

In [19]:
y.shape

(8848, 1)

In [20]:
for i in range(5):
    print(x[i])
    print(y[i])

[[0.61533666 0.61415212 0.61683292 0.6165212  0.61147132 0.61265586
  0.60592269 0.60118454 0.60473815 0.5973192  0.59264339 0.59052369
  0.5872818  0.58908978 0.59145885 0.58703242 0.580798   0.57842893
  0.58996259 0.59201995 0.59706983 0.60087282 0.59937656 0.60355362
  0.60592269 0.60679551 0.60473815 0.60885287 0.6026808  0.60174564
  0.6        0.59613466 0.59052369 0.60174564 0.60473815 0.60174564
  0.60561097 0.60916459 0.61359102 0.61209476 0.61564838 0.61477556
  0.61708229 0.61564838 0.61477556 0.61265586 0.61003741 0.60735661
  0.61359102 0.6165212  0.61770574 0.61502494 0.62331671 0.62244389
  0.62094763 0.62125935 0.61683292 0.61976309 0.62418953 0.62537406
  0.62861596 0.63154613 0.63185786 0.6292394  0.63216958 0.63453865
  0.63453865 0.63778055 0.63541147 0.63453865 0.63690773 0.63485037
  0.63690773 0.63865337 0.64164589 0.63983791 0.64133416 0.64014963
  0.63952618 0.63865337 0.63983791 0.64220698 0.64488778 0.64457606
  0.64513716 0.64600998 0.64488778 0.63778055 0.

In [21]:
arr_rand = np.random.rand(x.shape[0])
split = arr_rand < np.percentile(arr_rand, 90)

x_train = x[split]
y_train = y[split]
x_testeval = x[~split]
y_testeval = y[~split]

print(x_train.shape)
print(y_train.shape)
print(x_testeval.shape) 
print(y_testeval.shape)

arr_rand1 = np.random.rand(x_testeval.shape[0])
split1 = arr_rand1 < np.percentile(arr_rand1, 90)

x_test = x_testeval[split1]
y_test = y_testeval[split1]
x_eval = x_testeval[~split1]
y_eval = y_testeval[~split1]

print(x_test.shape) 
print(y_test.shape)
print(x_eval.shape) 
print(y_eval.shape)



(7963, 1, 100)
(7963, 1)
(885, 1, 100)
(885, 1)
(796, 1, 100)
(796, 1)
(89, 1, 100)
(89, 1)


In [22]:
for i in range(5):
    print(x_test[i])
    print(y_test[i])

[[0.580798   0.57842893 0.58996259 0.59201995 0.59706983 0.60087282
  0.59937656 0.60355362 0.60592269 0.60679551 0.60473815 0.60885287
  0.6026808  0.60174564 0.6        0.59613466 0.59052369 0.60174564
  0.60473815 0.60174564 0.60561097 0.60916459 0.61359102 0.61209476
  0.61564838 0.61477556 0.61708229 0.61564838 0.61477556 0.61265586
  0.61003741 0.60735661 0.61359102 0.6165212  0.61770574 0.61502494
  0.62331671 0.62244389 0.62094763 0.62125935 0.61683292 0.61976309
  0.62418953 0.62537406 0.62861596 0.63154613 0.63185786 0.6292394
  0.63216958 0.63453865 0.63453865 0.63778055 0.63541147 0.63453865
  0.63690773 0.63485037 0.63690773 0.63865337 0.64164589 0.63983791
  0.64133416 0.64014963 0.63952618 0.63865337 0.63983791 0.64220698
  0.64488778 0.64457606 0.64513716 0.64600998 0.64488778 0.63778055
  0.63597257 0.6361596  0.63410224 0.63410224 0.6388404  0.6388404
  0.64177057 0.6521197  0.65299252 0.64177057 0.63946384 0.63915212
  0.63528678 0.63584788 0.64002494 0.64326683 0.64

In [23]:
#x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1])) # Reshaping done for LSTM as it needs 3D input
#x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
#x_eval = x_eval.reshape((x_eval.shape[0], 1, x_eval.shape[1]))

In [24]:
#seerwe

In [25]:
def totaltestpredictionerror(y_test, y_testresult):
    ytestdiff = y_test - y_testresult
    ytestdiffabs = np.absolute(ytestdiff)
    sumoftesterrors = np.sum(ytestdiffabs) 
    y_test = None
    y_testresult = None
    ytestdiff = None
    ytestdiffabs =  None
    
    return sumoftesterrors

In [26]:
def totalevalpredictionerror(y_eval, y_evalresult):
    yevaldiff = y_eval - y_evalresult
    yevaldiffabs = np.absolute(yevaldiff)
    sumofevalerrors = np.sum(yevaldiffabs)
    y_eval = None
    y_evaltresult = None
    yevaldiff = None
    ytestdiffabs =  None
    
    return sumofevalerrors

In [27]:
def totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult):
    ytestdiff = y_test - y_testresult
    yevaldiff = y_eval - y_evalresult
    ytestdiffabs = np.absolute(ytestdiff)
    yevaldiffabs = np.absolute(yevaldiff)
    sumoferrors = np.sum(ytestdiffabs) + np.sum(yevaldiffabs)
    y_eval = None
    y_evaltresult = None
    y_test = None
    y_testresult = None
    ytestdiff = None
    yevaldiff = None
    ytestdiffabs =  None
    
    return sumoferrors

In [28]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape) 
print(y_test.shape)
print(x_eval.shape) 
print(y_eval.shape)

(7963, 1, 100)
(7963, 1)
(796, 1, 100)
(796, 1)
(89, 1, 100)
(89, 1)


In [29]:


regressor = Sequential()

regressor.add(LSTM(units = 800, return_sequences = True, input_shape = (x_train.shape[1], x_train.shape[2])))

regressor.add(LSTM(units = 800, return_sequences = True))

regressor.add(LSTM(units = 800))

regressor.add(Dense(units = 1))

regressor.compile(loss='mean_squared_error', optimizer='adam')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [32]:

with tf.device('gpu'):
    for i in range(60000):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028, verbose=0)
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        print("I-", i," Total - ", ytotaldiff)
        cutoff = 5.00
        if ytotaldiff < cutoff:
            print(i+0)
            regressor.save("EUR24H800Xhigh"+str(i+0)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
            
       

I- 0  Total -  9.364679423799558
I- 1  Total -  7.188758435837945
I- 2  Total -  8.590688780061601
I- 3  Total -  7.722002297684449
I- 4  Total -  9.513404103466971
I- 5  Total -  11.135590421826493
I- 6  Total -  7.507298251875012
I- 7  Total -  7.984727560671205
I- 8  Total -  9.903657781512932
I- 9  Total -  7.274156497868507
I- 10  Total -  14.415791784052209
I- 11  Total -  7.3738051819385015
I- 12  Total -  7.268064885276221
I- 13  Total -  8.267547486428887
I- 14  Total -  20.253928242300475
I- 15  Total -  7.361508632627796
I- 16  Total -  8.769090272720335
I- 17  Total -  8.332422747546614
I- 18  Total -  7.337808635526156
I- 19  Total -  7.445579839198655
I- 20  Total -  7.1754049061837115
I- 21  Total -  9.011795298892638
I- 22  Total -  8.267967903019501
I- 23  Total -  7.185503439177885
I- 24  Total -  17.21174197541809
I- 25  Total -  9.055897817914897
I- 26  Total -  6.836688708188825
I- 27  Total -  9.325498626000245
I- 28  Total -  6.766646736875139
I- 29  Total -  9.5

I- 237  Total -  7.328915728832995
I- 238  Total -  12.98545363612309
I- 239  Total -  5.854236572490371
I- 240  Total -  8.6722625062947
I- 241  Total -  5.859969198465937
I- 242  Total -  6.145964967966684
I- 243  Total -  28.309120603660062
I- 244  Total -  21.981732797087616
I- 245  Total -  13.818098371819696
I- 246  Total -  9.706236728587378
I- 247  Total -  6.41372024243014
I- 248  Total -  7.03644804630492
I- 249  Total -  5.987363211621074
I- 250  Total -  5.8939018591681025
I- 251  Total -  7.17132409365098
I- 252  Total -  9.89851576149318
I- 253  Total -  7.23935792945565
I- 254  Total -  6.048680162340617
I- 255  Total -  5.799006778789581
I- 256  Total -  5.7575693179544345
I- 257  Total -  5.7820592409060225
I- 258  Total -  9.491289036589045
I- 259  Total -  7.028615530886871
I- 260  Total -  5.772668522016668
I- 261  Total -  13.962526690692343
I- 262  Total -  8.709564723873402
I- 263  Total -  9.235338329376992
I- 264  Total -  7.336967051683241
I- 265  Total -  6.0

I- 471  Total -  9.329034119829604
I- 472  Total -  18.3117698832938
I- 473  Total -  8.949732532851982
I- 474  Total -  8.278011472326291
I- 475  Total -  7.3171801169019455
I- 476  Total -  5.937103174481916
I- 477  Total -  7.78923471940126
I- 478  Total -  7.789083979076306
I- 479  Total -  6.226002867144523
I- 480  Total -  7.953582751096904
I- 481  Total -  6.966197718170791
I- 482  Total -  8.747533239004348
I- 483  Total -  6.843450000309863
I- 484  Total -  5.346103793813703
I- 485  Total -  6.181896674900562
I- 486  Total -  9.690761441155889
I- 487  Total -  5.642394428627735
I- 488  Total -  13.7359161364467
I- 489  Total -  8.80548427607356
I- 490  Total -  5.385901837664049
I- 491  Total -  5.382641993407294
I- 492  Total -  5.504932925588166
I- 493  Total -  6.92311788048235
I- 494  Total -  6.109838493982174
I- 495  Total -  6.263764138501167
I- 496  Total -  5.762429947508231
I- 497  Total -  5.952317893861818
I- 498  Total -  23.904803939917997
I- 499  Total -  14.131

  Test --  4.370517590456176   Eval --  0.5529422716904167   Total -- ------------- 4.923459862146593
I- 702  Total -  7.441889659068208
I- 703  Total -  20.32137214542918
I- 704  Total -  10.28559251024839
I- 705  Total -  5.178638872630572
I- 706  Total -  5.797195520959873
I- 707  Total -  5.078570960376629
I- 708  Total -  6.611439543234121
I- 709  Total -  8.029848495237278
I- 710  Total -  5.088533985971506
I- 711  Total -  5.915710188563643
I- 712  Total -  4.947317166399778
712
  Test --  4.388017397747373   Eval --  0.559299768652405   Total -- ------------- 4.947317166399778
I- 713  Total -  6.936874402520639
I- 714  Total -  8.15860457898967
I- 715  Total -  5.299640756414429
I- 716  Total -  5.464166281853544
I- 717  Total -  6.36976931546991
I- 718  Total -  7.69972734252119
I- 719  Total -  6.682440790036052
I- 720  Total -  9.033879649520248
I- 721  Total -  6.621458518832082
I- 722  Total -  5.009201366675459
I- 723  Total -  5.451607317431027
I- 724  Total -  4.9590335

I- 858  Total -  13.40636055359523
I- 859  Total -  11.231034122708397
I- 860  Total -  7.677299989637322
I- 861  Total -  4.864978857200936
861
  Test --  4.311681362934539   Eval --  0.5532974942663962   Total -- ------------- 4.864978857200936
I- 862  Total -  6.743080344104981
I- 863  Total -  5.163666036123063
I- 864  Total -  5.145826337313702
I- 865  Total -  4.738754439175577
865
  Test --  4.207799605687065   Eval --  0.5309548334885124   Total -- ------------- 4.738754439175577
I- 866  Total -  17.12666465308597
I- 867  Total -  10.845797947130926
I- 868  Total -  4.960181421294173
868
  Test --  4.4133874401784565   Eval --  0.5467939811157172   Total -- ------------- 4.960181421294173
I- 869  Total -  7.579410226862301
I- 870  Total -  6.432619218279322
I- 871  Total -  5.274436529259443
I- 872  Total -  9.051785915896746
I- 873  Total -  5.05245674453175
I- 874  Total -  4.827394283323221
874
  Test --  4.284142224093035   Eval --  0.5432520592301864   Total -- -----------

...

I- 7784  Total -  3.1937133930270516
7784
  Test --  2.8536543094309197   Eval --  0.34005908359613213   Total -- ------------- 3.1937133930270516
I- 7785  Total -  3.2046472744751497
7785
  Test --  2.856574706662627   Eval --  0.34807256781252294   Total -- ------------- 3.2046472744751497
I- 7786  Total -  6.2852041126189055
I- 7787  Total -  5.108022463113619
I- 7788  Total -  3.742161182126516
7788
  Test --  3.331962991444563   Eval --  0.41019819068195296   Total -- ------------- 3.742161182126516
I- 7789  Total -  3.73817911900188
7789
  Test --  3.359245097458792   Eval --  0.37893402154308764   Total -- ------------- 3.73817911900188
I- 7790  Total -  3.6209339085362338
7790
  Test --  3.2513029807523455   Eval --  0.3696309277838885   Total -- ------------- 3.6209339085362338
I- 7791  Total -  3.1637096589937492
7791
  Test --  2.823858254954704   Eval --  0.33985140403904546   Total -- ------------- 3.1637096589937492
I- 7792  Total -  3.32018707431761
7792
  Test --  2.959

I- 7846  Total -  3.1737884516727846
7846
  Test --  2.8486836972973846   Eval --  0.3251047543754   Total -- ------------- 3.1737884516727846
I- 7847  Total -  4.286000717696977
7847
  Test --  3.856175015305619   Eval --  0.4298257023913583   Total -- ------------- 4.286000717696977
I- 7848  Total -  3.2725100389442616
7848
  Test --  2.915343526919888   Eval --  0.35716651202437344   Total -- ------------- 3.2725100389442616
I- 7849  Total -  3.7104140140469415
7849
  Test --  3.3090364496012454   Eval --  0.40137756444569606   Total -- ------------- 3.7104140140469415
I- 7850  Total -  3.473428467384314
7850
  Test --  3.0947480500785027   Eval --  0.37868041730581137   Total -- ------------- 3.473428467384314
I- 7851  Total -  3.472411880974743
7851
  Test --  3.1224124479472057   Eval --  0.34999943302753755   Total -- ------------- 3.472411880974743
I- 7852  Total -  3.2366660520025525
7852
  Test --  2.9057431091096846   Eval --  0.33092294289286794   Total -- ------------- 3.2

I- 7906  Total -  3.263774185584974
7906
  Test --  2.911624483247656   Eval --  0.3521497023373179   Total -- ------------- 3.263774185584974
I- 7907  Total -  3.1092019798749693
7907
  Test --  2.7856541074719505   Eval --  0.3235478724030185   Total -- ------------- 3.1092019798749693
I- 7908  Total -  3.181284111158515
7908
  Test --  2.841792920848681   Eval --  0.3394911903098341   Total -- ------------- 3.181284111158515
I- 7909  Total -  3.899358683750241
7909
  Test --  3.4881054430234077   Eval --  0.41125324072683345   Total -- ------------- 3.899358683750241
I- 7910  Total -  5.187355157889008
I- 7911  Total -  5.197821245972385
I- 7912  Total -  5.460048686566228
I- 7913  Total -  4.711779469088156
7913
  Test --  4.214456230268243   Eval --  0.49732323881991314   Total -- ------------- 4.711779469088156
I- 7914  Total -  5.1109551754675895
I- 7915  Total -  5.011448798958528
I- 7916  Total -  3.25508029255783
7916
  Test --  2.922685985285735   Eval --  0.3323943072720945

I- 7972  Total -  3.1712940817759194
7972
  Test --  2.8406659265706047   Eval --  0.33062815520531474   Total -- ------------- 3.1712940817759194
I- 7973  Total -  3.762487790858082
7973
  Test --  3.3597106073860115   Eval --  0.40277718347207037   Total -- ------------- 3.762487790858082
I- 7974  Total -  4.175604456559061
7974
  Test --  3.7574669172341473   Eval --  0.41813753932491293   Total -- ------------- 4.175604456559061
I- 7975  Total -  3.6062701199417577
7975
  Test --  3.2170495708684546   Eval --  0.38922054907330317   Total -- ------------- 3.6062701199417577
I- 7976  Total -  3.425444903546156
7976
  Test --  3.055277873244978   Eval --  0.3701670303011779   Total -- ------------- 3.425444903546156
I- 7977  Total -  3.170278345735893
7977
  Test --  2.8329824782369197   Eval --  0.33729586749897345   Total -- ------------- 3.170278345735893
I- 7978  Total -  4.075455310963017
7978
  Test --  3.671328714690581   Eval --  0.40412659627243563   Total -- ------------- 4.

RuntimeError: Problems closing file (file write failed: time = Tue Sep 29 20:51:03 2020
, filename = 'EUR24H800Xhigh7998.h5', file descriptor = 63, errno = 28, error message = 'No space left on device', buf = 0x559053647b10, total write size = 2048, bytes this sub-write = 2048, bytes actually written = 18446744073709551615, offset = 0)

In [31]:
bfghfgj
from tensorflow import keras
from tensorflow.keras import layers

regressor = load_model("EURUSD24H800X59228.h5")

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.005,
    decay_steps=1000,
    decay_rate=0.75,
    staircase=True)




NameError: name 'bfghfgj' is not defined

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.002)
regressor.compile(loss='mean_squared_error', optimizer=opt)

In [ ]:
gtc

In [ ]:
with tf.device('gpu'):
    for i in range(75000):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028, verbose=0)
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        #print("  Total -- -------------", ytotaldiff)
        cutoff = 3.51
        if ytotaldiff < cutoff:
            print(i+60)
            regressor.save("EURUSD24H"+str(i+60)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
            
       

In [ ]:

with tf.device('gpu'):
    for i in range(600):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 5.2:
            print(i)
            regressor.save("EURUSD24H"+str(i))+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:

with tf.device('gpu'):
    for i in range(600):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 4.7:
            print(i+600)
            regressor.save("EURUSD24H"+str(i+600)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:

with tf.device('gpu'):
    for i in range(600):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 4.5:
            print(i+1200)
            regressor.save("EURUSD24H"+str(i+1200)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:

with tf.device('gpu'):
    for i in range(1200):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 4.4:
            print(i+1800)
            regressor.save("EURUSD24H"+str(i+1800)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:
with tf.device('gpu'):
    for i in range(3000):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 4.08:
            print(i+3000)
            regressor.save("EURUSD24H"+str(i+3000)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:
with tf.device('gpu'):
    for i in range(6000):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 3.5348:
            print(i+18000)
            regressor.save("EURUSD24H"+str(i+18000)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:

with tf.device('gpu'):
    for i in range(20):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        #print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
        
        
        cutoff1 = 3.4281
        if ytotaldiff < cutoff1:
            print(" cutoff applied  -- " , cutoff1)
            print(i+60000)
            regressor.save("EURUSD24H"+str(i+60000)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
            cutoff1 = ytotaldiff
            print("new globval cutoff -- " , cutoff1)
       

In [ ]:
with tf.device('gpu'):
    for i in range(1):
        print(i)
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        regressor.save("EURUSDDAILYCLOSEM7.h5")

In [ ]:
y_testresult = regressor.predict(x_test)
y_evalresult = regressor.predict(x_eval)

In [ ]:
for i in range(5):
    print(y_testresult[i])
    print(y_test[i]) 

In [ ]:
for i in range(5):
    print(y_testresult[i])
    print(y_test[i]) 

In [ ]:
plot.plot(history.history['loss'], label='MAE (training data)')
#plot.plot(history.history['val_loss'], label='MAE (validation data)')

plot.show()

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [120, 80]
x = np.arange(len(y_testresult))
y=y_testresult.reshape(927)
y1=y_test.reshape(927)

yyy = y_testresult - y_test
ydif = yyy.reshape(927)

#plt.plot(x, y, label="John")

#plt.plot(x, y1, label="David")

plt.plot(x, y_testresult - y_test, label="Tom")



In [ ]:
good till here


In [ ]:
scrap
def ResNetFX(input_shape = (1,100)):
    """
    Arguments:
    input_shape -- shape 

    Returns:
    model -- a Model() instance in Keras
    """
    """
    size = 3200
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    X = Dense(size,  activation='relu')(X_input)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(1,  activation='linear')(X)
    """
    
    X_input = Input(input_shape)
    X = LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2]))
    X = Dense(1,  activation='linear')(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNetFX')

    return model


In [ ]:
scrapmodel = ResNetFX(input_shape = (100))

'''
model.compile(optimizer='adam', loss='mse', metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), 
    tf.keras.metrics.Recall(), tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(),
    tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives(), tf.keras.metrics.SensitivityAtSpecificity(),
    tf.keras.metrics.SpecificityAtSensitivity()])
'''
model.compile(optimizer='adam', loss = 'mean_squared_error', metrics=['mse'])

#model.fit(x_train, y_train, epochs = 10, batch_size = 8)



In [ ]:
scrap
gc.collect()
offset = 0

ytestdiff = np.zeros(600)
yevaldiff = np.zeros(600)
ytotaldiff = np.zeros(600)

#model = load_model("EURUSDDAILY"+str(offset-1)+".h5")
#model.compile(optimizer='adam', loss='mean_squared_error', 
              #metrics=[tf.keras.metrics.MeanAbsoluteError()])

with tf.device('gpu'):
    for i in range(600):
        history = model.fit(x_train, y_train, epochs = 1, batch_size = 64)
        model.save("EURUSD24H"+str(i+offset)+".h5")
        y_testresult = model.predict(x_test)
        y_evalresult = model.predict(x_eval)

        ytestdiff[i] = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff[i] = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff[i] = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)

        print("Iter#--", i, )
        print("  Test -- ", ytestdiff[i], "  Eval -- ", yevaldiff[i], "  Total -- ", ytotaldiff[i])


        gc.collect()
    #hist_df = pd.DataFrame(history.history) 
    # or save to csv: 
    #hist_csv_file = "EURUSDDailyhistory"+str(i+offset)+".csv"
    #with open(hist_csv_file, mode='w') as f:
        #hist_df.to_csv(f)

In [ ]:
scrap
for i in range(50):
    print(y_test[i])
    print(y_testresult[i])